<a href="https://colab.research.google.com/github/peterbabulik/Quantum-Alice-Bob-Demo/blob/main/Bob_Decrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# @title 🔓 QKD PART 2: BOB (Decoder)
# @markdown ### 🟢 Step 2: Measure and Decrypt
# @markdown **Fix:** Maps PauliZ eigenvalues (1, -1) back to Binary (0, 1)
# @markdown to ensure the key matches Alice's format exactly.

import subprocess
import sys
import numpy as np
import pickle
import warnings

warnings.filterwarnings("ignore")

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
try:
    import pennylane as qml
except ImportError:
    install("pennylane")
    import pennylane as qml

print("✅ Bob's Quantum Engine Ready.")

# ==========================================
# 1. RECEIVE TRANSMISSION
# ==========================================
try:
    with open("quantum_channel.pkl", "rb") as f:
        data = pickle.load(f)

    received_qubits = data["qubits"]
    alice_bases = data["bases"]

    with open("alice_private_key.pkl", "rb") as f:
        alice_bits_verification = pickle.load(f)

    print(f"\n📡 RECEIVED: {len(received_qubits)} Qubits from Alice.")
except FileNotFoundError:
    print("❌ ERROR: Files missing. Please upload 'quantum_channel.pkl' and 'alice_private_key.pkl'.")
    sys.exit()

# ==========================================
# 2. MEASURE QUBITS
# ==========================================
NUM_QUBITS = len(received_qubits)
bob_bases = np.random.randint(0, 2, NUM_QUBITS)
bob_measurements = []

dev = qml.device("default.qubit", wires=1, shots=1)

print("\n1️⃣  Bob is measuring qubits...")

for i in range(NUM_QUBITS):
    @qml.qnode(dev)
    def measure_qubit(state_vector, basis):
        qml.StatePrep(state_vector, wires=0)
        if basis == 1:
            qml.Hadamard(wires=0)
        return qml.sample(qml.PauliZ(0))

    # RAW MEASUREMENT: +1 or -1
    result = int(measure_qubit(received_qubits[i], bob_bases[i]))

    # --- FIX: MAP EIGENVALUES TO BITS ---
    # +1 means |0> (Bit 0)
    # -1 means |1> (Bit 1)
    bit = 0 if result == 1 else 1

    bob_measurements.append(bit)

print("   - Measurements complete.")

# ==========================================
# 3. SIFTING
# ==========================================
print("\n2️⃣  Sifting Keys...")

shared_key = []

for i in range(NUM_QUBITS):
    if alice_bases[i] == bob_bases[i]:
        shared_key.append(bob_measurements[i])

shared_key = np.array(shared_key)
print(f"   - Bases matched {len(shared_key)} times.")
print(f"   - 🔑 SHARED SECRET KEY GENERATED.")

# ==========================================
# 4. VERIFICATION & DECRYPTION
# ==========================================

# Verify
alice_key_part = []
for i in range(NUM_QUBITS):
    if alice_bases[i] == bob_bases[i]:
        alice_key_part.append(alice_bits_verification[i])
alice_key_part = np.array(alice_key_part)

if np.array_equal(shared_key, alice_key_part):
    print("\n✅ SUCCESS: Keys match perfectly.")
else:
    print("\n❌ FAILURE: Mismatch detected.")

# Decrypt Full Message
message_str = "HELLO_QUANTUM_WORLD"
msg_bits = []
for char in message_str:
    bin_char = format(ord(char), '08b')
    msg_bits.extend([int(b) for b in bin_char])

print(f"\n   Message Length: {len(msg_bits)} bits")
print(f"   Key Length:     {len(shared_key)} bits")

final_key = shared_key[:len(msg_bits)]

if len(final_key) < len(msg_bits):
    print(f"\n⚠️ STILL TOO SHORT! Need {len(msg_bits) - len(final_key)} more bits.")
else:
    # Encrypt
    encrypted_bits = np.bitwise_xor(msg_bits, final_key)
    # Decrypt
    decrypted_bits = np.bitwise_xor(encrypted_bits, final_key)

    decrypted_chars = []
    for i in range(0, len(decrypted_bits), 8):
        byte = decrypted_bits[i:i+8]
        char_code = int("".join(map(str, byte)), 2)
        decrypted_chars.append(chr(char_code))

    print(f"\n🔓 FULL DECRYPTED MESSAGE: {''.join(decrypted_chars)}")

✅ Bob's Quantum Engine Ready.

📡 RECEIVED: 500 Qubits from Alice.

1️⃣  Bob is measuring qubits...
   - Measurements complete.

2️⃣  Sifting Keys...
   - Bases matched 238 times.
   - 🔑 SHARED SECRET KEY GENERATED.

✅ SUCCESS: Keys match perfectly.

   Message Length: 152 bits
   Key Length:     238 bits

🔓 FULL DECRYPTED MESSAGE: HELLO_QUANTUM_WORLD
